In [ ]:
# # no need for this since it has nothing outside classes and functions
# print(__name__)
# if __name__ == "__main__" and hasattr(__builtins__,'__IPYTHON__') and ('google.colab' in str(get_ipython())):
#     from google.colab import drive
#     drive.mount('/content/drive')
#     %cd /content/drive/MyDrive/PressureReliefWorkArea/SummerWork/
#     !ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/PressureReliefWorkArea/SummerWork
AbstractModel.ipynb  DataProcessing.ipynb   RunAndEvaluation.ipynb
CNNModel.ipynb	     HelperFunctions.ipynb


In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from abc import ABC, abstractmethod



# -n does not work apparently??
    # __name__ == "__main__" even with -n.
# I commented under an issue logged on IPython's GitHub repository requesting this fix,
    # and I will be reposting the issue if it is not seen
    # (since that issue is very old and appears to have been left unseen so far)
%run -n DataProcessing.ipynb
# but apparently the ipynb module hates running %cd and !ls AND %run in an imported notebook???
    # I can't even import a second ipynb file from inside the first import...
# !pip install ipynb
# import ipynb
# from ipynb.fs.full.DataProcessing import *

In [ ]:
class SKJFModel(ABC):

    def tempfunc_use_unadjusted(self):
        cur_size = 2 # len(self.__loader.handler)
        hacky_tuple = (3, 2, 5, 6, 7, 8)
        dl_ind_list = list(self.__loader.current_dataloaders)
        dl_ind_list[0] = hacky_tuple.index(self.train_class_type) * cur_size
        print(f"Changed active dataloaders from indices {self.__loader.current_dataloaders} to indices {tuple(dl_ind_list)}")
        self.__loader.current_dataloaders = tuple(dl_ind_list)


    def tempfunc_prepare_loader(self):
        cur_size = 2
        hacky_tuple = (2, 5, 6, 7, 8)
        # dl_ind_list = list(self.__loader.current_dataloaders)
        for classification in hacky_tuple:
            self.__label_converter.set_label_type_conversion(3, classification)
            for i in range(cur_size):
                self.__loader.handler.recall_state(i, False)
                desc = list(self.__loader.handler.description)
                desc[5] = str(classification)
                self.__loader.handler.apply(lambda ds: SKDatasetHandler.save_data_and_labels([ds, ds.data, self.__label_converter.convert_label_type(ds.labels)]))
                self.__loader.handler.description = "".join(desc)
                # f"Type {classification}, Unchanged" # StatAndOth Repressed"
                self.__loader.handler.classification = classification
                self.__loader.handler.log_state(False)

    # this currently only truly works with my current setup in RunAndEvaluation.ipynb
        # if you alter enough that you can remove this, don't forget to alter change_io_type() as well
    def tempfunc_mount_loader(self, loader, train_classification, test_classification = None):
        self.__loader = loader
        self.train_class_type = train_classification
        self.__eval_class_type = test_classification

        # be weary,
        # I forget what I was going to say next, but you get the picture
        hacky = True
        if hacky:
            # no the stuff in this if block is not the only hacky stuff in this function
                # but it helps bring attention to it so it doesn't get forgotten
            print("WARNING: hacky code being used which is not as abstract/generalized as other functions")
        cur_size = 2 # len(self.__loader.handler)
        hacky_tuple = (3, 2, 5, 6, 7, 8)
        dl_ind_list = list(self.__loader.current_dataloaders)
        # don't do what the below comment says, it is more hacky than the current. Consider making stuff less hacky tho
        # add this back in later, and comment out the loop below it (at least for more general situations)
        # if only_type3:
        #     for i in indices:
        #         dl_ind_list[i] += cur_size
        # else:
        #     for i in indices:
        #         dl_ind_list[i] = cur_size * 2 - 1
        if not hacky:
            _, indices = self.__loader.get_dataloaders(None, "Train", True)
            # this is still hacky... I had to hack my own hack
            for i in indices:
                dl_ind_list[i] = (dl_ind_list[i] % 2) + len(self.__loader.handler) - cur_size
        dl_ind_list[0] = hacky_tuple.index(train_classification) * 2 + 1
        dl_ind_list[1] = hacky_tuple.index(train_classification) * 2
        print(f"Changed active dataloaders from indices {self.__loader.current_dataloaders} to indices {tuple(dl_ind_list)}")
        self.__loader.current_dataloaders = tuple(dl_ind_list)

    def __init__(self, sequence_length = None):
        super(SKJFModel, self).__init__()
        # self.loader = JFLoader(file_path, sequence_length)
        self.sequence_length = sequence_length
        self.train_class_type = None
        # self.train_loader = None
        self.__eval_class_type = None
        # self.__eval_loader = None
        self.__label_converter = None
        # self.__counter = 0

        self.__loader = None
        self.input_FS = None
        self.output_FS = None
        self.temp_tracker = 0

    # def get_eval_loader(self):
    #     if self.__eval_loader is None:
    #         return self.train_loader
    #     return self.__eval_loader

    def get_eval_class_type(self):
        if self.__eval_class_type is None:
            return self.train_class_type
        return self.__eval_class_type
    
    def get_loader(self):
        return self.__loader
    
    def get_outputs(self, inputs, feedback = False):
        # if self.__counter >= 5:
        #     raise RecursionError("SKJFModel.get_outputs() has detected what may be an infinite loop")
        
        if feedback:
            print("inputs:")
            print(inputs)

        # self.__counter += 1
        outputs = type(self).get_outputs(self, inputs, False)
        # self.__counter -= 1

        # if self.train_class_type == 3 and self.__eval_class_type == 5:
        #     # print(np.shape(outputs))
        #     outputs = self.__label_converter.convert_label_type(outputs, False, False, True, torch.Tensor, feedback)
        #     # print(np.shape(outputs))

        if feedback:
            print("outputs:")
            print(outputs)

        return outputs
        # raise NotImplementedError(f"SKJFModel.__get_outputs() is an abstract method and should be defined before used.")




    @abstractmethod
    def load_model(self, file_path, class_type):
        ...
        # raise NotImplementedError(f"SKJFModel.load_model() is an abstract method and should be defined before used.")

    @abstractmethod
    def train(self, feedback = False):
        ...
        # raise NotImplementedError(f"SKJFModel.train() is an abstract method and should be defined before used.")




    def change_io_specs(self, classification, *args, **kwargs):
        # if(class_type != 5 or self.train_class_type != 3):
        #     raise NotImplementedError("Yeah, this function was cobbled together. I strongly suggest putting other loading logic into place in places besides here before trying to work on it")
        # validate_conversion(self.train_class_type, class_type)

        # if args:
        #     repress_classes = args[0]
        #     if len(args) > 1:
        #         args = args[1:]
        #     else:
        #         args = []
        # else:
        #     repress_classes = kwargs.pop("repress_classes", True)

        # self.__eval_class_type = class_type
        # self.__eval_loader = JFSKLoader(SKFileNameHandler.get_filename_for_classtype(class_type), self.sequence_length, repress_classes, *args, **kwargs)
        self.__eval_class_type = classification
        # self.__loader = JFSKLoader(SKFileNameHandler.get_filename_for_classtype(class_type), self.sequence_length, repress_classes, *args, **kwargs)
        if self.__label_converter is None:
            self.__label_converter = SKLabelConverter(None, self.__loader.specifier_values)
        self.__label_converter.input_specifiers[SKDescriptors.CLASSIFICATION_TYPE_FS] = self.train_class_type
        self.__label_converter.input_specifiers[SKDescriptors.WITH_CLASS_NUMBER_FS] = 0
        self.__label_converter.set_label_type_conversion(self.train_class_type, classification)

        # right now this uses Type 3 dataset changes as the backbone. I feel like doing this
            # any other way would be more work than it's currently worth (at the time of typing this),
            # but feel free to take a crack at it
        only_type3 = True
        hacky = True
        if hacky:
            # no the stuff in this if block is not the only hacky stuff in this function
                # but it helps bring attention to it so it doesn't get forgotten
            print("WARNING: hacky code being used which is not as abstract/generalized as other functions")
        cur_size = 2 # len(self.__loader.handler)
        hacky_tuple = (3, 2, 5, 6, 7, 8)
        # dl_ind_list = list(self.__loader.current_dataloaders)
        # for i in range(cur_size):
        #     if self.__loader.handler.description.find('Type 3') == 0:
        #         desc = list(self.__loader.handler.description)
        #         desc[5] = str(classification)
        #         self.__loader.handler.recall_state(i, False)
        #         print(self.__loader.handler.apply(len, save = False))
        #         print(self.__loader.handler.apply(type, save = False))
        #         self.__loader.handler.apply(lambda ds: SKDatasetHandler.save_data_and_labels([ds, self.__label_converter.convert_label_type(ds.labels), ds.data]))
        #         self.__loader.handler.description = "".join(desc)
        #         # f"Type {classification}, Unchanged" # StatAndOth Repressed"
        #         self.__loader.handler.classification = classification
        #         self.__loader.handler.log_state(False)
        #     else:
        #         only_type3 = False
        dl_ind_list = list(self.__loader.current_dataloaders)
        # add this back in later, and comment out the loop below it (at least for more general situations)
        # if only_type3:
        #     for i in indices:
        #         dl_ind_list[i] += cur_size
        # else:
        #     for i in indices:
        #         dl_ind_list[i] = cur_size * 2 - 1
        if not hacky:
            _, indices = self.__loader.get_dataloaders(None, "Test", True)
            # this is still hacky... I had to hack my own hack
            for i in indices:
                dl_ind_list[i] = (dl_ind_list[i] % 2) + len(self.__loader.handler) - cur_size
        dl_ind_list[1] = hacky_tuple.index(classification) * 2
        print(f"Changed active dataloaders from indices {self.__loader.current_dataloaders} to indices {tuple(dl_ind_list)}")
        self.__loader.current_dataloaders = tuple(dl_ind_list)



    def load_data(self, file_path, *args, **kwargs):
        if args:
            repress_classes = args[0]
            if len(args) > 1:
                args = args[1:]
            else:
                args = []
        else:
            repress_classes = kwargs.pop("repress_classes", True)
        # self.train_loader = JFSKLoader(file_path, self.sequence_length, repress_classes, *args, **kwargs)
        # class_type = self.train_loader.specifier_values[SKDescriptors.CLASSIFICATION_TYPE_FS]
        # dataset states 0 and 1
        self.__loader = JFSKLoader(file_path, self.sequence_length, repress_classes, True, *args, **kwargs)
        class_type = self.__loader.specifier_values[SKDescriptors.CLASSIFICATION_TYPE_FS]
        # assertion
        SKDescriptors.validate_class_type(class_type)
        # needed definitions
        self.train_class_type = class_type
        if self.__label_converter is None:
            self.__label_converter = SKLabelConverter(None, self.__loader.specifier_values)
        self.tempfunc_prepare_loader()


    def evaluate(self, show_cm = True, save_cm_path = None, feedback = False):
        all_preds = []
        all_labels = []
        correct = 0
        total = 0
        first_iteration = True

        # counter = 0
        with torch.no_grad():   # Since we're not training, we don't need to calculate the gradients
            # print(self.__loader.handler.apply(lambda ds: np.shape(ds.data), save = False))
            for data in self.__loader.select("Test"): # self.get_eval_loader().test_dataloader:
                inputs, labels = data
                outputs = SKJFModel.get_outputs(self, inputs, feedback and first_iteration)
                # if self.train_class_type != self.get_eval_class_type():
                #     # torch.Tensor, 
                #     outputs = self.__label_converter.convert_label_type(outputs, False, False, True, first_iteration)
                _, predicted = torch.max(outputs.data, 1)
                if self.train_class_type != self.get_eval_class_type():
                    # torch.Tensor, 
                    predicted = self.__label_converter.convert_label_type(predicted, True, False, True, first_iteration)
                all_preds.extend(predicted.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                # if feedback and counter % 10 == 0:
                #     print("click")
                #     print(predicted.cpu().numpy())
                #     print(labels.cpu().numpy())
                # counter += 1
                if first_iteration:
                    first_iteration = False

        print('Accuracy of the network on the test data: %d %%' % (100 * correct / total))
        
        # self.jfm_plot_confusion_matrix(show_cm)
        if not show_cm and save_cm_path is None:
            return
        class_names = SKDescriptors.CTS_PER_TYPE[self.get_eval_class_type()] # [str(c) for c in SKDescriptors.CTS_PER_TYPE[self.class_type]]
        cm = confusion_matrix(all_labels, all_preds)

        fig, ax = plt.subplots(figsize=(10, 10))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
        plt.ylabel('Actual')
        plt.xlabel('Predicted')
        plt.title('Confusion Matrix')
        # saving needs to occur first, else we'll be left with a blank image
        if save_cm_path is not None:
            plt.savefig(save_cm_path)
        if show_cm:
            plt.show()